In [10]:
#!jupyter nbconvert --to='script' repair.ipynb

In [1]:
#special characters:

#NotASymbol
NAS = -1

#terminator
TERM = -2

In [2]:
def string_to_symbol_list(string, k):
    next_symbol = 0
    symbol_list = [TERM for x in range(k-1)]
    char_to_symbol_dict = {}

    for x in string:
        if x not in char_to_symbol_dict:
            char_to_symbol_dict[x] = next_symbol
            next_symbol += 1
        symbol_list.append(char_to_symbol_dict[x])

    symbol_list += [TERM for x in range(k-1)]
    return (symbol_list, char_to_symbol_dict, next_symbol)

In [3]:
#assuming sequence goes left to right 
class KTupleInfo:
    def __init__(self, count=0, last=-1, first=-1):
        self.count = count
        self.orig_count = 0 #stays 0 for pairs created in replacement
        self.last = last
        self.first = first
        
    @classmethod
    def get_print_lines(cls, items):
        lines = [f"{'Key':<6} {'Count':<6} {'Last':<6}",
                 "-" * 32]
        for i, item in items.items():
            lines.append(f"{str(i):<6} {item.count:<6} {item.last:<6}")
        return lines

    def __repr__(self):
        return f"KTupleInfo(count={self.count}, last={self.last})"

    def __eq__(self, other):
        if not isinstance(other, KTupleInfo):
            return NotImplemented
        return self.count == other.count and self.last == other.last

In [4]:
class SequenceElement:
    def __init__(self, symbol=None, pos=-1, prev_k_tuple=-1, next_k_tuple=-1):
        self.symbol = symbol
        self.pos = pos
        self.prev_k_tuple = prev_k_tuple
        self.next_k_tuple = next_k_tuple
        
    @classmethod
    def get_print_lines(cls, items):
        lines = [f"{'Index':<6} {'Symbol':<10} {'Pos':<6} {'PrevKTuple':<12} {'NextKTuple':<12}",
                 "-" * 50]
        for i, item in enumerate(items):
            lines.append(f"{str(i):<6} {str(item.symbol):<10} {item.pos:<6} {str(item.prev_k_tuple):<12} {str(item.next_k_tuple):<12}")
        return lines
        
    def __repr__(self):
        return (f"SequenceElement(symbol={self.symbol}, prev_k_tuple={self.prev_k_tuple}, pos={self.pos}, "
                f"next_k_tuple={self.next_k_tuple})")

    def __eq__(self, other):
        if not isinstance(other, SequenceElement):
            return NotImplemented
        return (self.symbol == other.symbol and
                self.pos == other.pos and
                self.prev_k_tuple == other.prev_k_tuple and
                self.next_k_tuple == other.next_k_tuple)

In [5]:
def construct_active_k_tuples_and_sequence(symbol_list, k):
    active_k_tuples = {}
    sequence = []

    for i in range(k - 1):
        elem = SequenceElement()
        elem.symbol = symbol_list[i]
        elem.prev_k_tuple = -1
        elem.pos = i
        elem.next_k_tuple = -1
        sequence.append(elem)

    for i in range(k - 1, len(symbol_list)):
        k_tuple = tuple(symbol_list[i - k + 1:i + 1])
        if k_tuple not in active_k_tuples:
            info = KTupleInfo()
            info.count = 1
            info.orig_count = 1
            info.last = i
            active_k_tuples[k_tuple] = info

            elem = SequenceElement()
            elem.symbol = symbol_list[i]
            elem.prev_k_tuple = -1
            elem.pos = i
            elem.next_k_tuple = -1
            sequence.append(elem)
        else:
            prev_index = active_k_tuples[k_tuple].last
            active_k_tuples[k_tuple].count += 1
            active_k_tuples[k_tuple].orig_count = active_k_tuples[k_tuple].count
            active_k_tuples[k_tuple].last = i

            elem = SequenceElement()
            elem.symbol = symbol_list[i]
            elem.pos = i
            elem.prev_k_tuple = prev_index
            sequence[prev_index].next_k_tuple = i
            sequence.append(elem)

    return sequence, active_k_tuples

In [6]:
# we use a dict based on priorities to manage priority queue
# no use for bothering with heapq

def construct_priority_queue(active_k_tuples):
    
    max_count = 0
    for v in active_k_tuples.values():
        if v.count > max_count:
            max_count = v.count

    priority_queue = [set() for i in range(max_count+1)]
            
    for k, v in active_k_tuples.items():
        priority_queue[v.count].add(k)
        
    return priority_queue

In [9]:
def get_prev_index(sequence, node): 
    #return -1 if not found
    if node.pos == 0:
        return -1 
    elif sequence[node.pos-1].symbol != NAS:
        return node.pos-1
    else:
        return sequence[node.pos-1].prev_k_tuple

def get_next_index(sequence, node):
    #return -1 if not found
    if node.pos == len(sequence)-1:
        return -1 
    elif sequence[node.pos+1].symbol != NAS:
        return node.pos+1
    else:
        return sequence[node.pos+1].next_k_tuple

def manual_prev_index(sequence, node):
    i = node.pos-1
    while sequence[i].symbol == -1:
        i -= 1
    return i

In [ ]:
import pdb
def check_linking(active_k_tuples, sequence, k_tuple):
    
    right = sequence[active_k_tuples[k_tuple].last]
    if right.prev_k_tuple == -1:
        print("count != 1 but prev_k_tuple == -1")
        import pdb
        pdb.set_trace()
    left = sequence[right.prev_k_tuple]
    
    while left.prev_k_tuple != -1:
        if left.next_k_tuple != right.pos:
            print("bad linking!!!")
            import pdb
            pdb.set_trace()
        right = left
        left = sequence[left.prev_k_tuple]
        
    if left.next_k_tuple != right.pos:
        print("bad last linking!!!")
        import pdb
        pdb.set_trace()

def check_pairs(active_k_tuples, sequence, k_tuple):
    
    node = sequence[active_k_tuples[k_tuple].last]
    while node.prev_k_tuple != -1:
        if (sequence[get_prev_index(sequence, node)].symbol, node.symbol) != k_tuple :
            print("bad ktuple!!!")
            import pdb
            pdb.set_trace()
        node = sequence[node.prev_k_tuple]
        
    if (sequence[get_prev_index(sequence, node)].symbol, node.symbol) != k_tuple :
            print("bad last ktuple!!!")
            import pdb
            pdb.set_trace()
        
def check_all_linking(active_k_tuples, sequence):
    for k_tuple in active_k_tuples:
        if active_k_tuples[k_tuple].count > 0:
            if active_k_tuples[k_tuple].count > 1:
                check_linking(active_k_tuples, sequence, k_tuple)
            check_pairs(active_k_tuples, sequence, k_tuple)

def ERROR(txt):
    print(txt)
    import pdb
    pbd.set_trace()

def check_entire_sequence(sequence, active_k_pairs):
    l_active_k_pairs = {}
    i = 1
    prev_node = sequence[0]
    while i < len(sequence)-1:
        node = sequence[i]
        if node.symbol != -1:
            k_tuple = (prev_node.symbol, node.symbol)
            if k_tuple not in l_active_k_pairs:
                l_active_k_pairs[k_tuple] = KTupleInfo()
                l_active_k_pairs[k_tuple].first = i
            else:
                #check pointers
                if node.prev_k_tuple != l_active_k_pairs[k_tuple].last:
                   ERROR("wrong pointer back")
                if sequence[l_active_k_pairs[k_tuple].last].next_k_tuple != i:
                    ERROR("wrong pointer forward")
            l_active_k_pairs[k_tuple].count += 1
            l_active_k_pairs[k_tuple].last = i
            
            i+=1
            
        else:
            if node.prev_k_tuple != i-1:
                ERROR("leftmost empty node bad left pointer")
            i_leftmost_non_empty = i-1
            i_rightmost_non_empty = node.next_k_tuple
            while i != i_leftmost_non_empty:
                if sequence[i].symbol != -1:
                    ERROR("empty sequence skipped something")
                i+=1
            if sequence[i].symbol == -1:
                ERROR("leftmost empty node bad right pointer")
            if sequence[i-1].next_k_tuple != i:
                ERROR("rightmost empty node bad right pointer")
            if sequence[i-1].prev_k_tuple != i_leftmost_non_empty:
                ERROR("rightmost empty node bad left pointer")

In [10]:
def replace_active_k_tuple(priority_queue, active_k_tuples, sequence, k_tuple, new_symbol,k):
    #works only for k=2
    #then need to come up with way to replace all the deleted tuples

    #keys of active_k_tuples which were changes anyhow
    changed_k_tuples = set()

    # assumming sequence = ... A B C D ... 
    # where (B, C) is the pair to be replaced
    # a_X means active_k_tuples[X]
    
    i_C = active_k_tuples[k_tuple].last
    while i_C != -1:
        #------------------replace single pair------------------
        
        C = sequence[i_C]
        B = sequence[get_prev_index(sequence, C)]
        A = sequence[get_prev_index(sequence, B)]
        D = sequence[get_next_index(sequence, C)]

        BC = (B.symbol, C.symbol)
        AB = (A.symbol, B.symbol)
        AE = (A.symbol, new_symbol)
        CD = (C.symbol, D.symbol)
        ED = (new_symbol, D.symbol)
        
        changed_k_tuples.add(BC)
        changed_k_tuples.add(AB)
        changed_k_tuples.add(AE)
        changed_k_tuples.add(CD)
        changed_k_tuples.add(ED)
            
        a_AB = active_k_tuples[AB]
        a_BC = active_k_tuples[BC]
        a_CD = active_k_tuples[CD]
        if AE not in active_k_tuples:
            active_k_tuples[AE] = KTupleInfo()
        a_AE = active_k_tuples[AE]
        if ED not in active_k_tuples:
            active_k_tuples[ED] = KTupleInfo()
        a_ED = active_k_tuples[ED]
        
        #update counts in active_k_tuples - decrement old
        a_BC.count -= 1
        a_AB.count -=1
        a_CD.count -=1
            
        #update counts in active_k_tuples - increment new
        a_AE.count +=1
        a_ED.count +=1
        
        
        #update list beginnings in active_k_tuples for old pairs 
        a_BC.last = C.prev_k_tuple
        # rightmost goes first!!!
        if a_CD.last == D.pos:
            #if this is the last C D in sequence:
            a_CD.last = D.prev_k_tuple
        if a_AB.last == B.pos:                                         
            #if this is the last A B in sequence:
            a_AB.last = B.prev_k_tuple
    
        #update list beginnings in active_k_tuples for new pairs 
        #this can be done when inserting them into active_k_tuples - this is just dumb, unoptimized version
        if a_AE.last  == -1:
            a_AE.last = C.pos
        if a_ED.last  == -1:
            a_ED.last = D.pos
            
            
        #update threading - old pairs
        if C.prev_k_tuple != -1:
            sequence[C.prev_k_tuple].next_k_tuple = -1
        #there never is C.next_k_tuple
        if B.prev_k_tuple != -1:
            sequence[B.prev_k_tuple].next_k_tuple = B.next_k_tuple
        if B.next_k_tuple != -1:
            sequence[B.next_k_tuple].prev_k_tuple = B.prev_k_tuple
        if D.prev_k_tuple != -1:
            sequence[D.prev_k_tuple].next_k_tuple = D.next_k_tuple
        if D.next_k_tuple != -1:
            sequence[D.next_k_tuple].prev_k_tuple = D.prev_k_tuple
            
        #update threading - new pairs 
            # WATCHOUT!!! PROBLEM WITH first WHEN NOT RUNNING THE CODE IN LOOP 
            # we can simulate the loop with the new pairs already existing in the sequence
            # but those do not have .first configured properly, instead they have -1
            # so newly added AE/ED do not point to AE/ED already present in the sequence
        C.next_k_tuple = a_AE.first
        if a_AE.first != -1:
            sequence[a_AE.first].prev_k_tuple = C.pos
        D.next_k_tuple = a_ED.first
        if a_ED.first != -1:
            sequence[a_ED.first].prev_k_tuple = D.pos
        C.prev_k_tuple = -1
        D.prev_k_tuple = -1 
        
        #update positions of the first elements of the lists
        a_AE.first = C.pos
        a_ED.first = D.pos
        #redundant - nobody cares about first element of the removed list 
        if a_BC.first == C.pos:
            a_BC.first = -1

        #manage empty spaces
        if sequence[B.pos-1].symbol != NAS:
            B.prev_k_tuple = B.pos-1
        else:
            B.prev_k_tuple = sequence[B.pos-1].prev_k_tuple
        
        if sequence[B.pos+1].symbol != NAS:
            B.next_k_tuple = B.pos+1
        else:
            B.next_k_tuple = sequence[B.pos+1].next_k_tuple

        #update rightmost/leftmost empty space
        if sequence[B.pos+1].symbol == NAS: 
            i_last_empty = sequence[B.pos+1].next_k_tuple -1
            sequence[i_last_empty].prev_k_tuple = B.prev_k_tuple
        if sequence[B.pos-1].symbol == NAS:
            i_last_empty = sequence[B.pos-1].prev_k_tuple +1
            sequence[i_last_empty].next_k_tuple = B.next_k_tuple
            
        
        
        #change symbols
        B.symbol = NAS
        C.symbol = new_symbol

        i_C = active_k_tuples[k_tuple].last

        check_all_linking(active_k_tuples, sequence)
        
    
    #update queues
    for changed in changed_k_tuples:
        
        #if it is not a new pair - new pairs are not in the queue yet 
        if active_k_tuples[changed].orig_count != 0:
            priority_queue[active_k_tuples[changed].orig_count].discard(changed)
        
        if active_k_tuples[changed].count == 0:
            del active_k_tuples[changed]
        else:
            priority_queue[active_k_tuples[changed].count].add(changed)
            active_k_tuples[changed].orig_count = active_k_tuples[changed].count 

    check_all_linking(active_k_tuples, sequence)



In [13]:
def read_from_sequence(sequence):
    res = []
    i = get_next_index(sequence, sequence[0])
    while i != -1:
        res.append(sequence[i].symbol)
        i = get_next_index(sequence, sequence[i])
    return res[:-1]

In [14]:
def repair(string, k):
    symbol_list, char_to_symbol_dict, next_symbol = string_to_symbol_list(string, k)
    sequence, active_k_tuples = construct_active_k_tuples_and_sequence(symbol_list, k)
    priority_queue = construct_priority_queue(active_k_tuples)
    
    symbol_to_k_tuple = {}
    
    i = len(priority_queue)-1
    while i > 1:
        while priority_queue[i]:
            to_replace = priority_queue[i].pop()
            replace_active_k_tuple(priority_queue, active_k_tuples, sequence, to_replace, next_symbol, k)
            symbol_to_k_tuple[next_symbol] = to_replace
            next_symbol += 1

            # check_all_linking(active_k_tuples, sequence)
        i -= 1

    return char_to_symbol_dict, symbol_to_k_tuple, read_from_sequence(sequence)

In [ ]:
def despair(char_to_symbol_dict, symbol_to_k_tuple, sequence):
    res = ""
    stack = []

    symbol_to_char_dict = {v:k for k,v in char_to_symbol_dict.items()}
    
    for x in sequence:
        stack.append(x)
        while stack:
            x = stack.pop()
            if x in symbol_to_k_tuple:
                stack.extend(reversed(symbol_to_k_tuple[x]))
            else:
                res += symbol_to_char_dict[x]
    return res